In [84]:
import json
import torch

In [85]:
annotations_file = 'graph_generation/small/train.json'

with open(annotations_file, 'r') as f:
    annotations = json.load(f)

def transform(annot):
    return {
        'labels': torch.Tensor(annot['labels']),
        'boxes': torch.Tensor(annot['boxes']),
    }

for k in annotations:
    annotations[k] = transform(annotations[k])

In [86]:
annotations['0']

{'labels': tensor([36., 36., 22., 29.]),
 'boxes': tensor([[0.2500, 0.3956, 0.0682, 0.0682],
         [0.7500, 0.6044, 0.0682, 0.0682],
         [0.2500, 0.3956, 0.0250, 0.0250],
         [0.7500, 0.6044, 0.0250, 0.0250]])}

In [90]:
from torchvision.ops import box_convert
from torchvision.utils import draw_bounding_boxes, make_grid
import torchvision.transforms as T
def display_annotations(img, boxes, device='cpu'):
    boxes = box_convert(boxes,'cxcywh', 'xyxy')
    img_bytes = T.functional.convert_image_dtype(img, torch.uint8)
    _,Y,X = img_bytes.shape
    scale = torch.Tensor([X,Y,X,Y]).to(device)
    boxes = torch.einsum('ij,j->ij',boxes,scale)
    boxes = boxes.to(torch.int)

    b = draw_bounding_boxes(img_bytes, boxes, width=3)
    b = b.to('cpu')
    return b

def display_all_annotations(imgs, labels):
    out = []
    print(len(imgs), len(labels))
    for img, label in zip(imgs, labels):
        # print(label)
        # print(label['boxes'])
        out.append(display_annotations(img, label['boxes']))


    return make_grid(out, nrow=5)


In [91]:
from PIL import Image

ns = [10,57,20,30,40,50,60,70,80,90]
# ns = [20]

imgs = []
labels = []
for n in ns:
    n = str(n)
    img = Image.open(f'graph_generation/small/train/{n}.png').convert('RGB')
    t = T.ToTensor()(img)
    imgs.append(t)
    labels.append(annotations[n])


In [93]:

res = display_all_annotations(imgs, labels)

res = T.ToPILImage()(res).show()

10 10


In [ ]:

def gen_test_segmentations(model, data_loader, device, thresh=0.7):
    imgs = []
    for image in data_loader:
        image = image.to(device)
        outputs = model(image)
        ps = outputs['pred_logits'].softmax(-1)[0, :, :-1]
        print(f"Max p: {ps.max()}")
        keep = ps.max(-1).values > thresh

        boxes = outputs['pred_boxes'][0,keep]
        boxes = box_convert(boxes, 'cxcywh', 'xyxy')
        img_bytes = T.functional.convert_image_dtype(image[0], torch.uint8)
        _,Y,X = img_bytes.shape
        scale = torch.Tensor([X,Y,X,Y]).to(device)
        boxes = torch.einsum('ij,j->ij',boxes,scale)
        boxes = boxes.to(torch.int)

        b = draw_bounding_boxes(img_bytes, boxes, width=3)
        b = b.to('cpu')
        imgs.append(b)

    grid = make_grid(imgs, nrow=2)

    return grid